In [1]:

import numpy as np
from services.gameManager import GameManager
from models.game import Game



0

In [3]:
gamemanager = GameManager(game)

In [4]:
!pwd

/home/schmijul/source/repos/robotarm


In [5]:
import moves

In [6]:
moves.legal_moves_generator(game.board, game.turn)

{(0, 0): array([0, 2, 2, 2, 2, 2, 2, 2, 2]),
 (0, 1): array([2, 0, 2, 2, 2, 2, 2, 2, 2]),
 (0, 2): array([2, 2, 0, 2, 2, 2, 2, 2, 2]),
 (1, 0): array([2, 2, 2, 0, 2, 2, 2, 2, 2]),
 (1, 1): array([2, 2, 2, 2, 0, 2, 2, 2, 2]),
 (1, 2): array([2, 2, 2, 2, 2, 0, 2, 2, 2]),
 (2, 0): array([2, 2, 2, 2, 2, 2, 0, 2, 2]),
 (2, 1): array([2, 2, 2, 2, 2, 2, 2, 0, 2]),
 (2, 2): array([2, 2, 2, 2, 2, 2, 2, 2, 0])}

In [7]:
import dlmodel as dlm

2022-08-05 23:07:57.388440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-05 23:07:57.388465: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
weak_mlp = dlm.build_model(10)
strong_mlp = dlm.build_model(100)

In [14]:
import training as t

0

In [16]:

game = Game()
game.toss()
weak_mlp = t.train(weak_mlp, game)[0]


game = Game()
game.toss()
strong_mlp = t.train(weak_mlp, game)[1]

1/1 [==============================] - 0s 17ms/step
[[0 0 0]
 [0 1 1]
 [0 1 1]]


ValueError: need at least one array to concatenate